In [3]:
%run code/import_libs.py
%run code/functions.py
#%run code/functions_v1.py
%run code/Build_Forms.py
%run code/initial_data_prep.py

%matplotlib inline

initial_pred = df.drop(columns=["price"]).copy()
initial_price = df[["price"]]

pred_fin, price_fin = transform_data(initial_pred, initial_price)

# Create OLS linear model

pred_int = sm.add_constant(pred_fin)
model = sm.OLS(price_fin,pred_int).fit()



coef_df=model.params.reset_index()
coef_df.columns=["Column","Value"]




In [4]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.876
Model:                            OLS   Adj. R-squared:                  0.875
Method:                 Least Squares   F-statistic:                     1702.
Date:                Thu, 31 Mar 2022   Prob (F-statistic):               0.00
Time:                        15:46:48   Log-Likelihood:                 5584.0
No. Observations:               21143   AIC:                        -1.099e+04
Df Residuals:                   21055   BIC:                        -1.029e+04
Df Model:                          87                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const             8.9231      0.190     46.951      0.000       8.551       9.296
sqft_living       0.4793      0.005     91.499      0.000       0.469       0.490
sqft_lot          0.0656      0.002     34.477      0.000       0.062       0.069
gra_4            -0.6584      0.190     -3.465      0.001      -1.031      -0.286
gra_5            -0.6868      0.187     -3.672      0.000      -1.053      -0.320
gra_6            -0.6155      0.187     -3.297      0.001      -0.981      -0.250
gra_7            -0.5410      0.187     -2.897      0.004      -0.907      -0.175
gra_8            -0.4434      0.187     -2.374      0.018      -0.809      -0.077
gra_9            -0.3142      0.187     -1.682      0.093      -0.680       0.052
gra_10           -0.2133      0.187     -1.141      0.254      -0.580       0.153
gra_11           -0.0874      0.187     -0.467      0.640      -0.454       0.279
gra_12            0.0503      0.188      0.267      0.789      -0.318       0.419
gra_13            0.3382      0.194      1.742      0.082      -0.042       0.719
zip_98002         0.0066      0.017      0.398      0.690      -0.026       0.039
zip_98003         0.0260      0.015      1.727      0.084      -0.004       0.055
zip_98004         1.1612      0.015     79.668      0.000       1.133       1.190
zip_98005         0.7661      0.018     43.547      0.000       0.732       0.801
zip_98006         0.6774      0.013     51.381      0.000       0.652       0.703
zip_98007         0.6760      0.019     36.195      0.000       0.639       0.713
zip_98008         0.6756      0.015     45.169      0.000       0.646       0.705
zip_98010         0.2434      0.021     11.407      0.000       0.202       0.285
zip_98011         0.4639      0.017     27.741      0.000       0.431       0.497
zip_98014         0.2808      0.020     14.236      0.000       0.242       0.319
zip_98019         0.3216      0.017     19.100      0.000       0.289       0.355
zip_98022         0.0578      0.016      3.620      0.000       0.026       0.089
zip_98023        -0.0194      0.013     -1.490      0.136      -0.045       0.006
zip_98024         0.4138      0.023     17.722      0.000       0.368       0.460
zip_98027         0.5357      0.014     39.130      0.000       0.509       0.563
zip_98028         0.4277      0.015     28.554      0.000       0.398       0.457
zip_98029         0.6293      0.015     42.902      0.000       0.601       0.658
zip_98030         0.0587      0.015      3.810      0.000       0.028       0.089
zip_98031         0.0930      0.015      6.144      0.000       0.063       0.123
zip_98032        -0.0141      0.020     -0.721      0.471      -0.052       0.024
zip_98033         0.8115      0.013     60.232      0.000       0.785       0.838
zip_98034         0.5589      0.013     43.681   

In [ ]:
coef_df[coef_df['Column'] == "wat_YES"]

In [6]:



def calculate_price (sqft_living, sqft_lot, basement, yr_built, zipcode, grade, waterfront, view , coef_df=coef_df):
    if view == '-':
        view='NONE'

    if waterfront == 'YES': waterfront='WATERFRONT'
    else: waterfront = 'NONE'

    b0,b1,b2,b3,b4,b5,b6,b7,b8 = get_coeff( yr_built, zipcode, grade, waterfront, view, coef_df)
    y=round( np.exp(b0 + b1*np.log(sqft_living) + b2*np.log(sqft_lot)  + b3 + b4 + b5 + b6 + b7 +b7+b8) )

#    print(y)
    return y,b0,b1,b2,b3,b4,b5,b6,b7,b8



abs_test_results=[]
test_results=[]
for i in range(1000):
    X=np.array(df[['sqft_living','sqft_lot', 'basement', 'yr_built','zipcode','grade','waterfront','view', 'price']].sample())[0]
    sqft_living, sqft_lot, basement, yr_built, zipcode, grade, waterfront, view ,price=X

#    b0,b1,b2,b3,b4,b5,b6,b7 = get_coeff( year=year, zipcode=zipcode, grade=grade, water=water, view=view, coef_df=coef_df)
#    y=round( np.exp(b0 + b1*np.log(sqft_living) + b2 + b3 + b4 + b5 + b6 + b7*np.log(sqft_lot)) )

    y,b0,b1,b2,b3,b4,b5,b6,b7,b8= calculate_price (sqft_living, sqft_lot, basement, yr_built, zipcode, grade, waterfront, view, coef_df=coef_df)

    miss_abs= np.abs(100-(y/price*100))
    miss= 100-(y/price*100)
    test_results.append(miss)
    abs_test_results.append(miss_abs)

avg_miss=sum(test_results)/len(test_results)
avg_miss_abs=sum(abs_test_results)/len(abs_test_results)
print (avg_miss, avg_miss_abs)


-2.087465717552185 14.817442595741587


In [7]:
print (X)
print(y,b0,b1,b2,b3,b4,b5,b6,b7,b8)
print (len(test_results))

[2620 4874 0 1999 98065 7 'NO' 'NONE' 470000.0]
439396 8.923123139453317 0.4793387911337554 0.06555019312216093 -0.032370683094030184 0 0.4372199204041556 -0.5409635874844052 0 -0.12332505374042213
1000


In [8]:
ui

NameError: name 'ui' is not defined

In [5]:
title=form_items[0].children[0]
medianW=form_items[1].children[0]
zipW=form_items[2].children[1]
yearW=form_items[2].children[2]
gradeW=form_items[3].children[1]
livingW=form_items[4].children[1]
livingW=form_items[4].children[2]
lotW=form_items[5].children[1]
viewW=form_items[6].children[1]
waterW=form_items[6].children[2]
#buttonW=form_items[5].children[0]

output = widgets.interactive_output(calculate_price, {
    'zipcode': zipW, 'yr_built':yearW, 'grade':gradeW,
    'sqft_living':livingW,'sqft_lot':lotW,'view':viewW,'waterfront':waterW } )

      
ui = widgets.VBox([form, output])

output.layout={'border': '3px solid green', 'width':'150px'}

#predictor=display(ui)


NameError: name 'calculate_price' is not defined